In [5]:
import bs4
import requests
import json
import pandas as pd
import warnings

warnings.filterwarnings(action="ignore")

with open("korean-books.csv", "a") as f:
    f.write("title,genres,author,rating,publisher,p-date,,pagesdescription,imgUrl\n")

subject_list = {
    "01":"소설", "03":"시/에세이", "05":"인문","13":"경제/경영", "15":"자기계발", 
    "19":"역사/문화", "21":"종교", "17":"정치/사회", "23":"예술/대중문화",
    "29":"과학", "26":"기술/공학", "33":"컴퓨터/IT",

    "38":"청소년", "11":"취미/실용/스포츠", "27":"외국어", "32":"여행",
    "07":"가정/육아", "08":"요리", "09":"건강"
}

img_src_url = "https://contents.kyobobook.co.kr/sih/fit-in/142x0/pdt/"
success = [200,299]
max_page = 11

for i in subject_list:
    print("="*100)
    print("start ",subject_list[i])
    for j in range(1,50,2):
        subjectNum = int(i)*100 + j
        if i==38:
            max_page = 6
        for page in range(1,max_page):
            url = f"https://product.kyobobook.co.kr/api/gw/pdt/category/all?page={page}&per=50&saleCmdtDvsnCode=KOR&saleCmdtClstCode={subjectNum:04d}&sort=sel"
            data = requests.get(url)
            if success[0]<=data.status_code and data.status_code<=success[1]:
                data = json.loads(data.content.decode("utf-8"))
                for n in range(50):
                    book = data["data"]["tabContents"][n]
                    title = book["cmdtName"]
                    author = book["chrcName"]
                    publisher = book["pbcmName"]
                    rating = book["revwRvgrAvg"] if book["revwRvgrAvg"]!=0 else None
                    p_date = book["rlseDate"]
                    description = book["inbukCntt"]
                    imgUrl = img_src_url + book["cmdtcode"] + ".jpg"
                    sub_url = "https://product.kyobobook.co.kr/detail/" + book["saleCmdtId"]
                    sub_data = requests.get(sub_url).content
                    sub_soup = bs4.BeautifulSoup(sub_data, "lxml")
                    genre_dic = {}
                    for genres in (sub_soup.find_all("li",class_="category_list_item")):
                        for genre in genres.find_all("a"):
                            genre_dic[genre.get_text()] = 1
                    genre_list = list(genre_dic.keys())
                    try:
                        pages = sub_soup.find("table",class_="tbl_row").tbody.get_text()\
                            .strip().replace(" ","").split("\n")[5].replace("쪽","")
                        with open("korean-books.csv", "a") as f:
                            f.write(f"\"{title}\",\"{genre_list}\",\"{author}\",\"{rating}\",\"{publisher}\",\"{p_date}\",\"{pages}\",\"{description}\",\"{imgUrl}\"\n")
                    except:
                        print(title,"ERROR")
                        pages = None
            else:
                pass

start  소설


KeyboardInterrupt: 

In [3]:
import pandas as pd
df = pd.read_csv("./korean-books.csv")
print(df.iloc[0])

ParserError: Error tokenizing data. C error: Expected 22 fields in line 34, saw 23


In [11]:
import pandas as pd

df = pd.DataFrame(columns=[
    "title_num",'author_num',"price"
])

import numpy as np
iter = 10
total_list = []
for i in range(10):
    dic = {}
    dic["title"] = np.random.randint(10)
    dic["author_num"] = np.random.randint(10)
    dic["price"] = np.random.randint(10)
    total_list.append(dic)
print(total_list)


[{'title': 2, 'author_num': 4, 'price': 2}, {'title': 4, 'author_num': 1, 'price': 1}, {'title': 7, 'author_num': 2, 'price': 0}, {'title': 2, 'author_num': 3, 'price': 5}, {'title': 4, 'author_num': 4, 'price': 8}, {'title': 8, 'author_num': 7, 'price': 0}, {'title': 0, 'author_num': 8, 'price': 2}, {'title': 9, 'author_num': 1, 'price': 6}, {'title': 7, 'author_num': 9, 'price': 2}, {'title': 9, 'author_num': 2, 'price': 6}]


In [13]:
import pandas as pd

data = pd.read_csv("./data/foreign-books.csv")
print(len(data))
print(data.iloc[0])
print(data["coverImg"].iloc[0])
print(data["description"].iloc[0])

52478
bookId                                       2767052-the-hunger-games
title                                                The Hunger Games
series                                            The Hunger Games #1
author                                                Suzanne Collins
rating                                                           4.33
description         WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...
language                                                      English
isbn                                                    9780439023481
genres              ['Young Adult', 'Fiction', 'Dystopia', 'Fantas...
characters          ['Katniss Everdeen', 'Peeta Mellark', 'Cato (H...
bookFormat                                                  Hardcover
edition                                                 First Edition
pages                                                             374
publisher                                            Scholastic Press
publishDate   

In [14]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
import pandas as pd

# create the Spark Session
spark = SparkSession.builder.appName("spark").getOrCreate()

# create the Spark Context
sc = spark.sparkContext

/home/ljj0512/.local/lib/python3.6/site-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


In [22]:
genres_list = (data["genres"].iloc[:].tolist())

In [32]:

rdd = sc.parallelize(genres_list)
print(rdd.take(1))
# print(rdd.take(2))
rdd1 = rdd.map(lambda line: line.replace("[","").replace("]","").replace("\'","").split(","))
print(type(rdd1.take(1)))

<class 'list'>
<class 'list'>


In [27]:
x = "[1,2,3]"
print(list(x))

['[', '1', ',', '2', ',', '3', ']']
